In [ ]:
# %pip install langgraph
# %pip install langchain
# %pip install langchain-openai
# %pip install langchain-cohere
# %pip install pinecone-client
# %pip install langchain-pinecone
# %pip install python-dotenv
# %pip install pandas 
# %pip install langchain_core
# %pip install langgraph-checkpoint-sqlite
# %pip install sqlalchemy

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.tools import tool
from langchain_cohere import ChatCohere
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from pinecone import Pinecone
import operator
import pandas as pd
import numpy as np
import os

import uuid
from contextlib import ExitStack

In [ ]:
## create an agent with these goals:
## 1. ingest user's idea
## 2. search the web for information on the largest companies in the space of their idea
## 3. return the top companies
## 4. 

load_dotenv()

# model = ChatCohere(model="command-r-plus")
model = ChatCohere(model="command-r-plus")

search_tool = TavilySearchResults(max_results=4) 

stack = ExitStack()
memory = stack.enter_context(SqliteSaver.from_conn_string(":memory:"))